# Messari Tokens Alert

In this notebook, we explore [Messari's API](https://messari.io/account/api) so we can build on top of it.

💡 Please add the API key in `.env`.

### Install dependencies

In [1]:
import os
import requests
import messari
import numpy as np
import pandas as pd
from pathlib import Path
from dotenv import load_dotenv
from urllib.parse import urljoin
from messari.messari import Messari

In [2]:
load_dotenv()
env_path = Path('.')/'.env'
load_dotenv(dotenv_path=env_path)
 
MESSARI_API = os.getenv("MESSARI_API")
MESSARI_URL = os.getenv("MESSARI_URL")
TOKENS_LIST = os.getenv("TOKENS_LIST")

if not (bool(MESSARI_API) and bool(MESSARI_URL) and bool(TOKENS_LIST)):
    raise Exception('Please add config to .env file')

In [3]:
# How many assets should be retrieved
ASSETS_LIMIT = 100

messari = Messari(MESSARI_API)

### Define util methods

In [4]:
def send_request(endpoint, less_log=False) -> dict:
    
    url = urljoin(MESSARI_URL, endpoint)
    
    if not less_log:
        print('Sending request to: {}'.format(url))
    
    try:
        r = requests.get(url)
        
    except requests.exceptions.HTTPError  as e:
        raise Exception('Error querying to {0}: {1}'.format(url, e.response.text))
    
    if r.status_code == 200:
        return r.json()
    else:
        raise Exception('Query failed - return code:{}.'.format(r.status_code))

In [5]:
def handle_KeyError(response, key) -> str:    
    try:
        return response[key]
    except KeyError as e:
         raise Exception('Response cannot be parsed.'.format(e))

In [6]:
def open_file(filename) -> list:
    try:
        with open(filename,'r') as f:
            return f.readlines()
    except IOError as e:
         raise Exception('Could not open {0}: {1}'.format(filename, e))

### Define API methods

In [7]:
def parse_assets_list(response):
    
    assets = pd.DataFrame(handle_KeyError(response, 'data'))
    assets["timestamp"] = pd.to_datetime(handle_KeyError(handle_KeyError(response, 'status'), 'timestamp'))    

    del assets["id"]
    del assets["serial_id"]
    del assets["contract_addresses"]
    del assets["_internal_temp_agora_id"]
    
    return assets

In [8]:
def get_assets():
    
    endpoint = '/api/v2/assets?fields&limit={}'.format(ASSETS_LIMIT)
    response = send_request(endpoint)

    return parse_assets_list(response)

In [9]:
def parse_asset_metrics(response):

    info = {}
    info['name'] = response['name']
    info['symbol'] = response['symbol']
    info['marketcap'] = response['marketcap']
    info['supply'] = response['supply']
    info['market_data'] = response['market_data']
    
    '''
    other data: 
    blockchain_stats_24_hours
    market_data_liquidity
    all_time_high
    token_sale_stats
    roi_data
    on_chain_data
    exchange_flows
    supply_activity
    '''
    
    return info

In [10]:
def get_asset_metrics(asset):
    
    endpoint = "https://data.messari.io/api/v1/assets/{}/metrics".format(asset)
    response = requests.get(endpoint).json()
    
    profile = handle_KeyError(response, 'data')
    
    del profile["id"]
    del profile["serial_id"]
    del profile["_internal_temp_agora_id"]
    del profile["contract_addresses"]
    del profile["alert_messages"]
    del profile["market_data"]['ohlcv_last_1_hour']
    del profile["developer_activity"]
    del profile["mining_stats"]
    del profile["misc_data"]
    del profile["reddit"]
    del profile["miner_flows"]

    return parse_asset_metrics(response['data'])

### Let's get a list of assets

In [11]:
assets_list = get_assets()
assets_list

Sending request to: https://data.messari.io/api/v2/assets?fields&limit=100


,symbol,name,slug,metrics,profile,timestamp
0,ACU,Acuity,acuity,"{'market_data': {'price_usd': None, 'price_btc...","{'general': {'overview': {'is_verified': None,...",2022-02-23 02:33:49.653547618+00:00
1,None,Rigoblock,rigoblock,"{'market_data': {'price_usd': None, 'price_btc...","{'general': {'overview': {'is_verified': None,...",2022-02-23 02:33:49.653547618+00:00
2,DAXX,DaxxCoin,daxxcoin,"{'market_data': {'price_usd': None, 'price_btc...","{'general': {'overview': {'is_verified': None,...",2022-02-23 02:33:49.653547618+00:00
3,POS,PokemonSpace,pokemonspace,"{'market_data': {'price_usd': None, 'price_btc...","{'general': {'overview': {'is_verified': None,...",2022-02-23 02:33:49.653547618+00:00
4,None,AgentMile,agentmile,"{'market_data': {'price_usd': None, 'price_btc...","{'general': {'overview': {'is_verified': None,...",2022-02-23 02:33:49.653547618+00:00
...,...,...,...,...,...,...
95,DICE,BetDice,betdice,"{'market_data': {'price_usd': None, 'price_btc...","{'general': {'overview': {'is_verified': None,...",2022-02-23 02:33:49.653547618+00:00
96,TKY,THEKEY,thekey-1,"{'market_data': {'price_usd': None, 'price_btc...","{'general': {'overview': {'is_verified': None,...",2022-02-23 02:33:49.653547618+00:00
97,DBALL,DrakeBall Token [OLD],drakeball-token,"{'market_data': {'price_usd': None, 'price_btc...","{'general': {'overview': {'is_verified': None,...",2022-02-23 02:33:49.653547618+00:00
98,BSDS,Basis Dollar Share,basis-dollar-share,"{'market_data': {'price_usd': None, 'price_btc...","{'general': {'overview': {'is_verified': None,...",2022-02-23 02:33:49.653547618+00:00


At any time we can look at one asset with:

In [12]:
assets_list.at[1, 'metrics']

{'market_data': {'price_usd': None,
  'price_btc': None,
  'price_eth': None,
  'volume_last_24_hours': None,
  'real_volume_last_24_hours': None,
  'volume_last_24_hours_overstatement_multiple': None,
  'percent_change_usd_last_1_hour': None,
  'percent_change_btc_last_1_hour': None,
  'percent_change_eth_last_1_hour': None,
  'percent_change_usd_last_24_hours': None,
  'percent_change_btc_last_24_hours': None,
  'percent_change_eth_last_24_hours': None,
  'ohlcv_last_1_hour': None,
  'ohlcv_last_24_hour': None,
  'last_trade_at': None},
 'marketcap': {'rank': None,
  'marketcap_dominance_percent': None,
  'current_marketcap_usd': None,
  'y_2050_marketcap_usd': None,
  'y_plus10_marketcap_usd': None,
  'liquid_marketcap_usd': None,
  'volume_turnover_last_24_hours_percent': None,
  'realized_marketcap_usd': None,
  'outstanding_marketcap_usd': None},
 'supply': {'y_2050': None,
  'y_plus10': None,
  'liquid': None,
  'circulating': None,
  'y_2050_issued_percent': None,
  'annual_inf

### Let's get an asset metric

In [13]:
asset = 'btc'

get_asset_metrics(asset)

{'name': 'Bitcoin',
 'symbol': 'BTC',
 'marketcap': {'rank': 1,
  'marketcap_dominance_percent': 44.462297436241116,
  'current_marketcap_usd': 715776571808.0922,
  'y_2050_marketcap_usd': 792075462582.5609,
  'y_plus10_marketcap_usd': 780340272903.1763,
  'liquid_marketcap_usd': 716113096968.5267,
  'volume_turnover_last_24_hours_percent': 0.7450513018848468,
  'realized_marketcap_usd': 457041344492.9346,
  'outstanding_marketcap_usd': 703550978541.0667},
 'supply': {'y_2050': 20986335.65,
  'y_plus10': 20675407.41,
  'liquid': 18973684.36,
  'circulating': 18964768,
  'y_2050_issued_percent': 90.40970599362352,
  'annual_inflation_percent': 1.7716864770274867,
  'stock_to_flow': 56.44339520374888,
  'y_plus10_issued_percent': 91.76933727953079,
  'supply_revived_90d': 18964642.18175567},
 'market_data': {'price_usd': 37772.37162471851,
  'price_btc': 1,
  'price_eth': 14.412089236923386,
  'volume_last_24_hours': 33177595609735.297,
  'real_volume_last_24_hours': 5347741437.313511,
 

### Let's print it cool

In [14]:
def print_market_data_asset(data):
    
    marketcap = data['market_data']
    
    print('{}:'.format(data['name']))
    print('    Price USD: {}'.format(marketcap['price_usd']))
    print('    Price BTC: {}'.format(marketcap['price_btc']))
    print('    Price ETH: {}'.format(marketcap['price_eth']))
    print('    Volume last 24h: {}'.format(marketcap['volume_last_24_hours']))
    print('    Real volume last 24h: {}'.format(marketcap['real_volume_last_24_hours']))
    print('    Percent change USD last 24h: {}'.format(marketcap['percent_change_usd_last_24_hours']))
    print()

In [15]:
def print_market_data():
    
    print('🚨 MARKET DATA FOR ASSETS') 
    print()
    print()
    
    assets = open_file(TOKENS_LIST)
    
    for asset in assets:
        print_market_data_asset(get_asset_metrics(asset.strip()))

In [16]:
print_market_data()

🚨 MARKET DATA FOR ASSETS


Bitcoin:
    Price USD: 37773.829922406076
    Price BTC: 1
    Price ETH: 14.410349989147493
    Volume last 24h: 33177595869306.633
    Real volume last 24h: 5347968571.600628
    Percent change USD last 24h: 2.0054246237467876

Ethereum:
    Price USD: 2621.0941797339387
    Price BTC: 0.06939138716052895
    Price ETH: 1
    Volume last 24h: 10523633925.519892
    Real volume last 24h: 4493493696.309488
    Percent change USD last 24h: 1.6303906736220595

SushiSwap:
    Price USD: 3.353905812979642
    Price BTC: 8.879506778147972e-05
    Price ETH: 0.0012796088533185497
    Volume last 24h: 45992230.48154507
    Real volume last 24h: 43780074.76536869
    Percent change USD last 24h: 3.5622435569541033

Alchemix:
    Price USD: 134.3494803970169
    Price BTC: 0.003533758715881081
    Price ETH: 0.051041103190211455
    Volume last 24h: 0
    Real volume last 24h: 1015543.7534190215
    Percent change USD last 24h: 1.9212725458438087

Index Cooperative:


## Using Messari API methods

In [19]:
messari.get_all_assets()

{'status': {'elapsed': 177, 'timestamp': '2022-02-23T02:34:32.708813886Z'},
 'data': [{'id': '1e31218a-e44e-4285-820c-8282ee222035',
   'serial_id': 6057,
   'symbol': 'BTC',
   'name': 'Bitcoin',
   'slug': 'bitcoin',
   'contract_addresses': None,
   '_internal_temp_agora_id': '9793eae6-f374-46b4-8764-c2d224429791',
   'metrics': {'market_data': {'price_usd': 37790.293936974886,
     'price_btc': 1,
     'price_eth': 14.410998941005456,
     'volume_last_24_hours': 33216170645258.38,
     'real_volume_last_24_hours': 5352642589.084223,
     'volume_last_24_hours_overstatement_multiple': 6205.564838757765,
     'percent_change_usd_last_1_hour': -0.7445734457665969,
     'percent_change_btc_last_1_hour': 0,
     'percent_change_eth_last_1_hour': -0.811680416854362,
     'percent_change_usd_last_24_hours': 2.0498844733451165,
     'percent_change_btc_last_24_hours': -0.2948721781691277,
     'percent_change_eth_last_24_hours': -0.051505710715582215,
     'ohlcv_last_1_hour': {'open': 38

In [39]:
messari.get_all_markets()

{'status': {'elapsed': 42, 'timestamp': '2022-02-23T02:39:16.723194858Z'},
 'data': [{'id': '00029600-7335-4a57-94ff-87beffaf8b97',
   'exchange_id': '65d7db09-764a-41a8-ac63-50a2d296f874',
   'base_asset_id': 'e23572af-c451-4bed-a085-361f9c207589',
   'quote_asset_id': '4d7b4b96-ecd1-47f7-b8ec-c054dd78b7ed',
   'class': 'spot',
   'trade_start': '2021-09-04T06:24:12Z',
   'trade_end': None,
   'version': 1,
   'excluded_from_price': False,
   'exchange_name': 'Uniswap (v3)',
   'exchange_slug': 'uniswap-v3',
   'base_asset_symbol': 'QBU',
   'quote_asset_symbol': 'WETH',
   'pair': 'QBU-WETH',
   'price_usd': None,
   'vwap_weight': 1,
   'volume_last_24_hours': None,
   'has_real_volume': True,
   'deviation_from_vwap_percent': None,
   'last_trade_at': None},
  {'id': '00030799-f441-4b8e-a327-889537d4c62a',
   'exchange_id': '19e0fd2a-2226-4552-80da-749a66953588',
   'base_asset_id': 'a3b64831-4288-48bd-a3e7-eb4f77f6f740',
   'quote_asset_id': '938dec5f-c021-42a3-95da-6cece8497dc9',